# Face Recognition

In [ ]:
import cv2
import numpy as np
def generate_dataset():
    face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

    def face_extractor(img):
        #gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces =face_classifier.detectMultiScale(img,1.3,5)
        
        if faces is ():
            return None
        for (x,y,w,h) in faces:
            x=x-10
            y=y-10
            cropped_face= img[y:y+h+50, x:x+w+50]
            
        return cropped_face


    cap = cv2.VideoCapture(0)

    name = "Shreeyukta"
    count =0   
    
    while True:
        
        ret, frame=cap.read()
        if face_extractor(frame) is not None:
            count +=1
            face=cv2.resize(face_extractor(frame),(400,400))
            #face=cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
            #save file in specified direction
            file_name_path="data/" + str(name)+"."+str(count)+".jpg"
            cv2.imwrite(file_name_path,face)
        
            cv2.putText(face,str(count),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
            cv2.imshow("Cropped_Face", face)
        else:
            print("Face not Found")
            pass
        if cv2.waitKey(1)==13 or count==300:#13 is the ASCI charater of enter jasle chahi infinite loopma cameralai lana baat rokxa
            break
    cap.release()
    cv2.destroyAllWindows()
    
    print("Collecting samples is completed")
    
#generate_dataset()

In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from keras.utils import get_file
from keras.utils import data_utils
import os
os.environ['KERAS_HOME'] = 'C:/Users/user/OneDrive/Desktop/Face reconition by cnn'


# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/Face reconition by cnn/Datasets/Training'
valid_path = '/content/drive/MyDrive/Face reconition by cnn/Datasets/Test'

# add preprocessing layer to the front of VGG
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False
  

  
  # useful for getting number of classes
folders = glob('/content/drive/MyDrive/Face reconition by cnn/Datasets/Training/*')
  

# our layers - you can add more if you want
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()

# tell the model what cost and optimization method to use
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Face reconition by cnn/Datasets/Training',
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Face reconition by cnn/Datasets/Test',
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical')

'''r=model.fit_generator(training_set,
                         samples_per_epoch = 8000,
                         nb_epoch = 10,
                         validation_data = test_set,
                         nb_val_samples = 2000)'''

# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)
# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# accuracies
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

plt.savefig('AccVal_acc')

import tensorflow as tf

from keras.models import load_model

model.save('facefeatures_new_model.h5')

In [1]:
from PIL import Image
from keras.applications.vgg16 import preprocess_input
import base64
from io import BytesIO
import json
import random
import cv2
from keras.models import load_model
import numpy as np
import mysql.connector

from keras.preprocessing import image
model = load_model('facefeatures_new_model.h5')

# Connect with database
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database="Face_Recognition"
)

mycursor = mydb.cursor()

# Loading the cascades
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    #gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(img, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

# Doing some Face Recognition with the webcam
video_capture = cv2.VideoCapture(0)
while True:
    _, frame = video_capture.read()
    #canvas = detect(gray, frame)
    #image, face =face_detector(frame)
    
    face=face_extractor(frame)
    if type(face) is np.ndarray:
        face = cv2.resize(face, (224, 224))
        im = Image.fromarray(face, 'RGB')
           #Resizing into 128x128 because we trained the model with this image size.
        img_array = np.array(im)
                    #Our keras model used a 4D tensor, (images x height x width x channel)
                    #So changing dimension 128x128x3 into 1x128x128x3 
        img_array = np.expand_dims(img_array, axis=0)
        pred = model.predict(img_array)
        print(pred)
                     
        name="None matching"
        
        if(pred[0][4]>0.5):
            name='Shreeyukta'
            
            # Insert attendance record into the database
            sql = "INSERT INTO attendance (name, rollno, email, address) VALUES (%s, %s, %s, %s)"
            val = ("Shreyukta", '23', "shree098gmail.com", "Kapan,Kathmandu")
            mycursor.execute(sql, val)
            mydb.commit()

            

            
        cv2.putText(frame,name, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
    else:
        cv2.putText(frame,"No face found", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
video_capture.release()
cv2.destroyAllWindows()


<>:35: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:35: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\user\AppData\Local\Temp\ipykernel_11864\1914323111.py:35: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


1/1 [==============================] - 1s 640ms/step
[[5.706703e-16 4.367649e-21 1.000000e+00 0.000000e+00 8.812744e-34]]
1/1 [==============================] - 0s 418ms/step
[[1.7021751e-21 8.6794983e-25 1.0000000e+00 0.0000000e+00 0.0000000e+00]]
1/1 [==============================] - 0s 376ms/step
[[1.01492e-28 0.00000e+00 1.00000e+00 0.00000e+00 0.00000e+00]]
1/1 [==============================] - 0s 364ms/step
[[6.4914696e-34 0.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00]]
1/1 [==============================] - 0s 393ms/step
[[0. 0. 1. 0. 0.]]
1/1 [==============================] - 0s 374ms/step
[[2.7469275e-37 0.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00]]
1/1 [==============================] - 0s 374ms/step
[[1.1686862e-22 4.5697989e-32 1.0000000e+00 0.0000000e+00 0.0000000e+00]]
1/1 [==============================] - 0s 362ms/step
[[1.0692882e-13 9.9970478e-01 2.3834882e-11 0.0000000e+00 2.9519625e-04]]
1/1 [==============================] - 0s 369ms/step
[

1/1 [==============================] - 0s 352ms/step
[[2.1709233e-18 2.7588031e-23 1.0000000e+00 0.0000000e+00 1.0130857e-37]]
1/1 [==============================] - 0s 362ms/step
[[9.9984574e-01 1.4920881e-20 1.5424892e-04 0.0000000e+00 1.2731871e-35]]
1/1 [==============================] - 0s 354ms/step
[[9.9998081e-01 1.0463416e-14 1.9200636e-05 0.0000000e+00 8.6377930e-37]]
1/1 [==============================] - 0s 373ms/step
[[1.0386695e-12 4.7636288e-20 1.0000000e+00 0.0000000e+00 2.3326767e-28]]
1/1 [==============================] - 0s 358ms/step
[[1.1663177e-13 3.1885961e-29 1.0000000e+00 0.0000000e+00 0.0000000e+00]]
1/1 [==============================] - 0s 367ms/step
[[4.3446857e-20 0.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00]]
1/1 [==============================] - 0s 356ms/step
[[1.6804832e-22 0.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00]]
1/1 [==============================] - 0s 356ms/step
[[1.9419254e-17 0.0000000e+00 1.0000000e+00 0.0000000e+00 

IntegrityError: 1062 (23000): Duplicate entry '23' for key 'PRIMARY'